# EDINETから有価証券報告書を取得するコード

コードのほとんどはこのウェブサイトから引用しました。\
https://zenn.dev/robes/articles/b7be539b65ee9d

In [ ]:
#xbrlパーサのインストール
!pip install edinet_xbrl

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.2 MB/s eta 0:00:00
  Created wheel for edinet_xbrl: filename=edinet_xbrl-0.2.0-py2.py3-none-any.whl size=10844 sha256=3d0648d0f60de8fa947e7b0e1b2d1182001e140d60c16c17c107c9fe001dc9d7
  Stored in directory: /root/.cache/pip/wheels/ef/28/ee/40c43a477ad1a6990efb30c760ecb93a4cfde3a25954de938f
  Created wheel for python-xbrl: filename=python_xbrl-1.1.1-py2.py3-none-any.whl size=10613 sha256=5a16034f4b58bfadbd0a59556e52e189ee9be9607884cec57d1fd1505dbff5eb
  Stored in directory: /root/.cache/pip/wheels/22/b6/99/6564765ea62e55df43f2906a5bfda311cd476ef958a28bf850
  Created wheel for ordereddict: filename=ordereddict-1.1-py3-none-any.whl size=3510 sha256=466ea6d16a889f77f15c53d62e23d149c2247a4d68100fb3da81e7d22ac0ec14
  Stored in directory:

In [ ]:
#xbrlパーサのインポート
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
parser = EdinetXbrlParser()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from bs4 import BeautifulSoup
import zipfile
import io
import requests
import pandas as pd
from tqdm.notebook import tqdm
import time
import glob
import re

In [ ]:
# EDINETAPIのトークンを取得する
from google.colab import userdata

edinet_api_key = userdata.get('EDINET_API_KEY')

In [ ]:
base_dir = "/content/drive/MyDrive/directory/"

In [ ]:
import requests
import datetime
import pandas as pd
import time
import zipfile
import warnings
from tqdm.notebook import tqdm
import io
import os

# 警告を特定のものに限定する
warnings.filterwarnings('ignore', category=DeprecationWarning)

class GetDocid:
    # 1 コンストラクタ・日付リストの作成
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self.day_list = self.create_day_list()

    def create_day_list(self):
        day_list = []
        period = (self.end_date - self.start_date).days
        for d in range(period + 1):
            day = self.start_date + datetime.timedelta(days=d)
            day_list.append(day)
        return day_list

    # 2 レポートリストの作成
    def create_report_list(self):
        report_list = []
        for day in self.day_list:
            url = "https://api.edinet-fsa.go.jp/api/v2/documents.json"
            params = {"date": day, "type": 2, "Subscription-Key": edinet_api_key}
            try:
                res = requests.get(url, params=params)
                res.raise_for_status()
                json_data = res.json()
                time.sleep(3)  # APIのレート制限に従って調整
            except requests.RequestException as e:
                print(f"Request failed: {e}")
                continue

            for result in json_data.get("results", []):
                if result["ordinanceCode"] == "010" and result["formCode"] == "030000":
                    report_list.append({
                        '会社名': result["filerName"],
                        '書類名': result["docDescription"],
                        'docID': result["docID"],
                        '証券コード': result["secCode"],
                        'ＥＤＩＮＥＴコード': result["edinetCode"],
                        '決算期': result["periodEnd"],
                        '提出日': day
                    })
        return report_list

    def create_docid_df(self, base_dir):
        # ファイルパスを動的に設定
        zip_path = "/content/Edinetcode.zip"
        extract_dir = "/content/Edinetcode"
        df_info_path = f"{extract_dir}/EdinetcodeDlInfo.csv"
        output_dir = f"{base_dir}/csv"  # 出力ディレクトリ
        output_path = f"{output_dir}/edinet_df.csv"

        # 出力ディレクトリを作成（存在しない場合）
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print(f"Created directory: {output_dir}")

        # edinetcodelistを取得する
        !wget -O {zip_path} https://disclosure2dl.edinet-fsa.go.jp/searchdocument/codelist/Edinetcode.zip  # 変更: -O オプションでファイル名を指定

        # zipファイルのダウンロードと展開
        try:
            with zipfile.ZipFile(zip_path) as zip_f:
                zip_f.extractall("Edinetcode")
        except zipfile.BadZipFile:
            print("Failed to unzip file")
            return None

        df_info = pd.read_csv(df_info_path, encoding="cp932", skiprows=[0])
        df_report = pd.DataFrame(self.create_report_list())
        df_info = df_info[["ＥＤＩＮＥＴコード", "提出者業種"]]
        merged_df = pd.merge(df_report, df_info, how="inner", on="ＥＤＩＮＥＴコード")
        merged_df.to_csv(output_path, index=False)

        return merged_df


In [ ]:
#提出期間の指定
start_date = datetime.date(2019,5,24)
end_date = datetime.date(2019,6,23)
#2024/11/20-2024/9/20-2024/7/20-2024/7/1-2024/6/20-2024/6/1-2024/5/1-2024/4/1-2024/3/20-2024/3/1-2024/2/1-2024/1/1-
#2023/12/1-2023/11/1-2023/9/1-2023/7/1-6/30-6/28-6/26-6/1-5/1-4/1

#2023/5/31-3/30-2/1-2022/10/1-4/1-2/1-2021/10/1-7/1-6/25-6/20-4/20-2020/11/20-5/20-1/20-2019/9/20-7/20-6/27-6/24

In [ ]:
#クラスのインスタンス化
gd = GetDocid(start_date,end_date)

In [ ]:
#docIDのデータフレーム取得
%%time
edinet_df = gd.create_docid_df(base_dir)

--2025-02-05 05:07:16--  https://disclosure2dl.edinet-fsa.go.jp/searchdocument/codelist/Edinetcode.zip
Resolving disclosure2dl.edinet-fsa.go.jp (disclosure2dl.edinet-fsa.go.jp)... 13.107.246.40, 2620:1ec:bdf::40
Connecting to disclosure2dl.edinet-fsa.go.jp (disclosure2dl.edinet-fsa.go.jp)|13.107.246.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 550931 (538K) [application/octet-stream]
Saving to: ‘/content/Edinetcode.zip’

/content/Edinetcode 100%[===================>] 538.02K   566KB/s    in 1.0s    

2025-02-05 05:07:19 (566 KB/s) - ‘/content/Edinetcode.zip’ saved [550931/550931]

CPU times: user 287 ms, sys: 21.3 ms, total: 309 ms
Wall time: 15.2 s


In [ ]:
edinet_df

,会社名,書類名,docID,証券コード,ＥＤＩＮＥＴコード,決算期,提出日,提出者業種
0,日本空調サービス株式会社,有価証券報告書－第56期(平成30年4月1日－平成31年3月31日),S100G4LT,46580,E04973,2019-03-31,2019-06-24,サービス業
1,オーデリック株式会社,有価証券報告書－第80期(平成30年4月1日－平成31年3月31日),S100G1PI,68890,E02026,2019-03-31,2019-06-24,電気機器
2,三信電気株式会社,有価証券報告書－第68期(平成30年4月1日－平成31年3月31日),S100G1N4,81500,E02671,2019-03-31,2019-06-24,卸売業
3,株式会社ゼンショーホールディングス,有価証券報告書－第37期(平成30年4月1日－平成31年3月31日),S100G0QR,75500,E03291,2019-03-31,2019-06-24,小売業
4,第一生命ホールディングス株式会社,有価証券報告書－第117期(平成30年4月1日－平成31年3月31日),S100G1X8,87500,E06141,2019-03-31,2019-06-24,保険業
...,...,...,...,...,...,...,...,...
954,エア・ウォーター株式会社,有価証券報告書－第19期(平成30年4月1日－平成31年3月31日),S100G867,40880,E00792,2019-03-31,2019-06-26,化学
955,株式会社インテージホールディングス,有価証券報告書－第47期(平成30年4月1日－平成31年3月31日),S100GAGD,43260,E05239,2019-03-31,2019-06-26,情報・通信業
956,幼児活動研究会株式会社,有価証券報告書－第47期(平成30年4月1日－平成31年3月31日),S100GA60,21520,E05691,2019-03-31,2019-06-26,サービス業
957,地盤ネットホールディングス株式会社,有価証券報告書－第11期(平成30年4月1日－平成31年3月31日),S100G8DY,60720,E27059,2019-03-31,2019-06-26,サービス業


In [ ]:
print(edinet_df.shape)

(959, 8)


In [ ]:
docid_list = edinet_df["docID"].tolist()

In [ ]:
from bs4 import BeautifulSoup
import zipfile
import io
import requests
import pandas as pd
from tqdm.notebook import tqdm
import time
import glob
import re

In [ ]:
import time
import requests
import zipfile
import io
from tqdm import tqdm

class GetReportFromEdinet:

    def __init__(self, docid_list, batch_size=100):
        self.docid_list = docid_list
        self.batch_size = batch_size

    def get_xbrl_file(self):
        # バッチ処理
        for i in range(0, len(self.docid_list), self.batch_size):
            batch = self.docid_list[i:i + self.batch_size]
            print(f"Processing batch {i // self.batch_size + 1}...")

            for docid in tqdm(batch):
                # 書類取得APIのエンドポイント
                url = f"https://api.edinet-fsa.go.jp/api/v2/documents/{docid}"
                print(url)
                time.sleep(3)

                # 書類取得APIのパラメータ
                params = {"type": 1, "Subscription-Key": edinet_api_key}
                res = requests.get(url, params=params, verify=False)

                # ファイルへの出力
                print(res.status_code)
                if res.status_code == 200:
                    # レスポンスからZIPファイルを読み込む
                    with zipfile.ZipFile(io.BytesIO(res.content)) as z:
                        # ZIPファイル内のすべてのファイルをループ処理
                        for file in z.namelist():
                            if file.startswith("XBRL/PublicDoc/") and file.endswith(".xbrl"):
                                # .xbrlファイルを見つけたら、それをディスクに書き込む
                                z.extract(file, path=f'/content/{docid}/')


In [ ]:
grfe = GetReportFromEdinet(docid_list)

In [ ]:
%%time
grfe.get_xbrl_file()

Processing batch 1...


  0%|          | 0/100 [00:00<?, ?it/s]

https://api.edinet-fsa.go.jp/api/v2/documents/S100G4LT


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
  1%|          | 1/100 [00:05<09:50,  5.96s/it]

200
https://api.edinet-fsa.go.jp/api/v2/documents/S100G1PI


...

200
https://api.edinet-fsa.go.jp/api/v2/documents/S100G8DY


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
 98%|█████████▊| 58/59 [05:35<00:05,  5.57s/it]

200
https://api.edinet-fsa.go.jp/api/v2/documents/S100G9XC


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
100%|██████████| 59/59 [05:40<00:00,  5.77s/it]

200
CPU times: user 56.4 s, sys: 11.1 s, total: 1min 7s
Wall time: 1h 32min 30s


In [ ]:
import glob
import pandas as pd
import re
from bs4 import BeautifulSoup
from tqdm import tqdm

class GetTextFromXBRL:

    def __init__(self, keys, context_ref, docid_list, batch_size=100):
        self.keys = keys
        self.context_ref = context_ref
        self.docid_list = docid_list
        self.batch_size = batch_size

    def parse_xbrl(self, xbrl_path):
        try:
            return parser.parse_file(xbrl_path), None
        except Exception as e:
            return None, str(e)

    def extract_data_from_xbrl(self, edinet_xbrl_object, key):
        if not edinet_xbrl_object:
            return None
        data = edinet_xbrl_object.get_data_by_context_ref(key, self.context_ref)
        if data:
            value = data.get_value()
            if value:
                return BeautifulSoup(value, 'html.parser').get_text(strip=True)
        return None

    # EDINETコードの取得
    def extract_dynamic_code(self, xbrl_path):
        pattern = r'jpcrp030000-asr-001_E(\d{5})'
        match = re.search(pattern, xbrl_path)
        if match:
            return "E" + match.group(1)
        else:
            return None

    def get_text_data_generic(self):
        columns = ["docID"] + list(self.keys.keys())
        extracted_data_list = []

        # バッチ処理
        for i in range(0, len(self.docid_list), self.batch_size):
            batch = self.docid_list[i:i + self.batch_size]
            print(f"Processing batch {i // self.batch_size + 1}...")

            for _docid in tqdm(batch):
                xbrl_path = f'/content/{_docid}/XBRL/PublicDoc/*.xbrl'
                xbrl_paths = glob.glob(xbrl_path)
                if xbrl_paths:
                    xbrl_path = xbrl_paths[0]
                    edinet_xbrl_object, error = self.parse_xbrl(xbrl_path)
                    if error:
                        print(f"Error parsing XBRL for {_docid}: {error}")
                        continue

                    row_data = [_docid]
                    for key_text in self.keys.values():
                        extracted_text = self.extract_data_from_xbrl(edinet_xbrl_object, key_text)
                        row_data.append(extracted_text)

                    extracted_data_list.append(row_data)

            print(f"Batch {i // self.batch_size + 1} completed.")

        return pd.DataFrame(extracted_data_list, columns=columns)


In [ ]:
keys = {
    "事業の内容":"jpcrp_cor:DescriptionOfBusinessTextBlock",
    "経営方針":"jpcrp_cor:BusinessPolicyBusinessEnvironmentIssuesToAddressEtcTextBlock" #経営方針、経営環境および対処すべき課題等
    #"事業等のリスク":"jpcrp_cor:BusinessRisksTextBlock", #事業等のリスク
    #"従業員の状況":"jpcrp_cor:InformationAboutEmployeesTextBlock", #従業員の状況
}

context_ref = "FilingDateInstant"

In [ ]:
gtfb = GetTextFromXBRL(keys,context_ref,docid_list)

In [ ]:
#text_dfの初期化
text_df = pd.DataFrame()

In [ ]:
%%time
text_df = gtfb.get_text_data_generic()

Processing batch 1...


  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/xbrl/xbrl.py:26: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(fh, "lxml")
100%|██████████| 100/100 [00:37<00:00,  2.68it/s]


Batch 1 completed.
Processing batch 2...


100%|██████████| 100/100 [00:40<00:00,  2.45it/s]


Batch 2 completed.
Processing batch 3...


...

Batch 8 completed.
Processing batch 9...


100%|██████████| 100/100 [00:39<00:00,  2.54it/s]


Batch 9 completed.
Processing batch 10...


100%|██████████| 59/59 [00:26<00:00,  2.26it/s]

Batch 10 completed.
CPU times: user 5min 59s, sys: 3.62 s, total: 6min 3s
Wall time: 6min 35s


In [ ]:
text_df.head(5)

,docID,事業の内容,経営方針
0,S100G4LT,３【事業の内容】当社グループは、当社（日本空調サービス株式会社）、子会社16社（国内７社、海...,１【経営方針、経営環境及び対処すべき課題等】当社グループは、「お客様に安心感を与える最適な環...
1,S100G1PI,３ 【事業の内容】当社グループは、当社及び連結子会社４社で構成され、各種照明器具の製造及び販...,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...
2,S100G1N4,３【事業の内容】当社の企業集団は、当社、子会社14社及び関連会社１社で構成されており、当社グ...,１【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、有価証券報告書提出日現...
3,S100G0QR,３ 【事業の内容】当社グループは、当社及び子会社98社の計99社により構成されており、フード...,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...
4,S100G1X8,３ 【事業の内容】当社グループは2019年３月31日現在、当社(保険持株会社）及び当社の関係...,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...


In [ ]:
def merge_df(df1, df2,how = "left"):
    #最初の2つのDataFrameを結合
    merged_df = pd.merge(df1, df2, on='docID', how=how)

    return merged_df

#merged_dfの初期化
merged_df = pd.DataFrame()

merged_df = merge_df(edinet_df,text_df)
print(merged_df.head(5))
print(merged_df.shape)

                 会社名                                  書類名     docID  証券コード  \
0       日本空調サービス株式会社   有価証券報告書－第56期(平成30年4月1日－平成31年3月31日)  S100G4LT  46580   
1         オーデリック株式会社   有価証券報告書－第80期(平成30年4月1日－平成31年3月31日)  S100G1PI  68890   
2           三信電気株式会社   有価証券報告書－第68期(平成30年4月1日－平成31年3月31日)  S100G1N4  81500   
3  株式会社ゼンショーホールディングス   有価証券報告書－第37期(平成30年4月1日－平成31年3月31日)  S100G0QR  75500   
4   第一生命ホールディングス株式会社  有価証券報告書－第117期(平成30年4月1日－平成31年3月31日)  S100G1X8  87500   

  ＥＤＩＮＥＴコード         決算期         提出日  提出者業種  \
0    E04973  2019-03-31  2019-06-24  サービス業   
1    E02026  2019-03-31  2019-06-24   電気機器   
2    E02671  2019-03-31  2019-06-24    卸売業   
3    E03291  2019-03-31  2019-06-24    小売業   
4    E06141  2019-03-31  2019-06-24    保険業   

                                               事業の内容  \
0  ３【事業の内容】当社グループは、当社（日本空調サービス株式会社）、子会社16社（国内７社、海...   
1  ３ 【事業の内容】当社グループは、当社及び連結子会社４社で構成され、各種照明器具の製造及び販...   
2  ３【事業の内容】当社の企業集団は、当社、子会社14社及び関連会社１社で構成されており、当社グ...   
3  ３ 【事業の内容】当社グループは、当社及び子会社98社

In [ ]:
import os
import pandas as pd

# 既存のCSVファイルパス
file_path = '/content/drive/My Drive/merged_df9.csv'

# ファイル名のバージョンを増加させる関数
def get_next_file_name(file_path):
    """
    ファイル名に番号を付加して次の保存ファイル名を生成する。
    :param file_path: 元のファイルパス
    :return: 新しいファイルパス
    """
    dir_name, base_name = os.path.split(file_path)
    file_name, ext = os.path.splitext(base_name)

    # 番号付きファイル名のパターンをチェック
    version = 1
    while True:
        new_file_name = f"{file_name}_v{version}{ext}"
        new_file_path = os.path.join(dir_name, new_file_name)
        if not os.path.exists(new_file_path):  # 存在しない場合に使用
            return new_file_path
        version += 1

# 条件を満たさない行を削除する関数
def remove_empty_rows(df, columns):
    """
    指定されたカラムが空白の場合に行を削除する。
    :param df: pandas DataFrame
    :param columns: チェックするカラムのリスト
    :return: フィルタリング後のDataFrame
    """
    for column in columns:
        df = df[df[column].notna() & (df[column].str.strip() != "")]
    return df

# 結合する新しいデータ (仮の例: merged_df)
# merged_dfは他の処理で準備されたDataFrameと仮定
# merged_df = pd.DataFrame({...})

# CSVファイルが既に存在する場合の処理
if os.path.exists(file_path):
    # 既存のデータを読み込む
    existing_df = pd.read_csv(file_path)
    # 指定カラムの空白行を削除
    existing_df = remove_empty_rows(existing_df, ['事業の内容', '経営方針'])
    # 新しいデータも空白行を削除
    merged_df = remove_empty_rows(merged_df, ['事業の内容', '経営方針'])
    # 新しいデータを結合
    combined_df = pd.concat([existing_df, merged_df])
    # 'docID'で重複を削除（最新のデータを優先）
    combined_df.drop_duplicates(subset=['docID'], keep='last', inplace=True)

    # ファイル番号を増加させた新しいファイルパスを取得
    new_file_path = get_next_file_name(file_path)
else:
    # ファイルが存在しない場合は新しいデータを使用
    merged_df = remove_empty_rows(merged_df, ['事業の内容', '経営方針'])
    combined_df = merged_df
    new_file_path = file_path

# 結果をCSVに保存
combined_df.to_csv(new_file_path, index=False, encoding='utf-8')
print(f"データが {new_file_path} に保存されました。")


データが /content/drive/My Drive/merged_df9_v1.csv に保存されました。
